# Import library

In [1]:
import importlib
import torch
import pickle
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pNN_Split as pnn
import random
import config
import evaluation as E
import training
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# Random Seed

In [2]:
seed = 0

# Device

In [3]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda:0')
device = 'cpu'
device

'cpu'

# Prepare data

## Datasets

In [4]:
ds1 = 'Seeds'

In [5]:
ds2 = 'Iris'

## Load data

In [6]:
datapath = os.path.join(os.getcwd(), 'Datasets',
                        'dataset_processed', f'Dataset_{ds1}.p')
with open(datapath, 'rb') as f:
    dataset = pickle.load(f)
X1 = dataset['X'].float()
y1 = dataset['y']

E1, N_features1, N_class1 = X1.shape[0], X1.shape[1], torch.max(torch.unique(y1)).item()+1
X1.shape, y1.shape, E1, N_features1, N_class1

(torch.Size([210, 7]), torch.Size([210]), 210, 7, 3)

In [7]:
datapath = os.path.join(os.getcwd(), 'Datasets',
                        'dataset_processed', f'Dataset_{ds2}.p')
with open(datapath, 'rb') as f:
    dataset = pickle.load(f)
X2 = dataset['X'].float()
y2 = dataset['y']

E2, N_features2, N_class2 = X2.shape[0], X2.shape[1], torch.max(torch.unique(y2)).item()+1
X2.shape, y2.shape, E2, N_features2, N_class2

(torch.Size([150, 4]), torch.Size([150]), 150, 4, 3)

## Pseudo-electrical Signal

In [8]:
X1 = X1 - torch.min(X1, axis=0)[0]
X1 = X1 / (torch.max(X1, axis=0)[0])
torch.min(X1), torch.max(X1)

(tensor(0.), tensor(1.))

In [9]:
X2 = X2 - torch.min(X2, axis=0)[0]
X2 = X2 / (torch.max(X2, axis=0)[0])
torch.min(X2), torch.max(X2)

(tensor(0.), tensor(1.))

In [10]:
# split
train_rate = 0.6
valid_rate = 0.2
test_rate = 0.2

E_train1 = int(E1*train_rate)
E_valid1 = int(E1*valid_rate)
E_test1 =  E1 - E_train1 - E_valid1

random.seed(config.data_split_seed);
np.random.seed(config.data_split_seed);
torch.manual_seed(config.data_split_seed);

idx = torch.randperm(E1)
X1 = X1[idx,:]
y1 = y1[idx]

X_train1, X_rest1 = X1[:E_train1,:],      X1[E_train1:,:]
X_valid1, X_test1 = X_rest1[:E_valid1,:], X_rest1[E_valid1:,:]

y_train1, y_rest1 = y1[:E_train1],      y1[E_train1:]
y_valid1, y_test1 = y_rest1[:E_valid1], y_rest1[E_valid1:]

X_train1.shape, y_train1.shape, X_valid1.shape, y_valid1.shape, X_test1.shape, y_test1.shape

(torch.Size([126, 7]),
 torch.Size([126]),
 torch.Size([42, 7]),
 torch.Size([42]),
 torch.Size([42, 7]),
 torch.Size([42]))

In [11]:
E_train2 = int(E2 * train_rate)
E_valid2 = int(E2 * valid_rate)
E_test2 =  E2 - E_train2 - E_valid2

random.seed(config.data_split_seed);
np.random.seed(config.data_split_seed);
torch.manual_seed(config.data_split_seed);

idx = torch.randperm(E2)
X2 = X2[idx,:]
y2 = y2[idx]

X_train2, X_rest2 = X2[:E_train2,:],      X2[E_train2:,:]
X_valid2, X_test2 = X_rest2[:E_valid2,:], X_rest2[E_valid2:,:]

y_train2, y_rest2 = y2[:E_train2],      y2[E_train2:]
y_valid2, y_test2 = y_rest2[:E_valid2], y_rest2[E_valid2:]

X_train2.shape, y_train2.shape, X_valid2.shape, y_valid2.shape, X_test2.shape, y_test2.shape

(torch.Size([90, 4]),
 torch.Size([90]),
 torch.Size([30, 4]),
 torch.Size([30]),
 torch.Size([30, 4]),
 torch.Size([30]))

In [12]:
X_trains = [X_train1, X_train2]
y_trains = [y_train1, y_train2]

In [13]:
X_valids = [X_valid1, X_valid2]
y_valids = [y_valid1, y_valid2]

In [14]:
X_tests = [X_test1, X_test2]
y_tests = [y_test1, y_test2]

# PNN

In [15]:
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);

## Define

In [16]:
num_in = []
num_in.append(N_features1)
num_in.append(N_features2)
num_in

[7, 4]

In [17]:
num_out = []
num_out.append(N_class1)
num_out.append(N_class2)
num_out

[3, 3]

In [18]:
topology_hidden = [5,5]

In [19]:
SuperPNN = pnn.SuperPNN(num_in, num_out, topology_hidden)

In [20]:
for n,p in SuperPNN.named_parameters():
    print(n)

theta_common_ 0
models.0.model.Input_Layer.theta_
models.0.model.Hiddel_Layer 0.theta_individual_
models.0.model.Output_Layer.theta_
models.1.model.Input_Layer.theta_
models.1.model.Hiddel_Layer 0.theta_individual_
models.1.model.Output_Layer.theta_


In [21]:
optimizer = torch.optim.Adam(SuperPNN.parameters(), lr=config.lr)

In [22]:
SuperPNN, train_losses, valid_losses, train_accs, valid_accs = training.train_spnn(SuperPNN, X_trains, y_trains, X_valids, y_valids, optimizer, pnn.LOSSFUNCTION)

The ID for this training is 1656941121.
| epoch:    0 | train acc: 34.4 | loss: 1.52e+02 | valid acc: 27.6 | loss: 5.09e+01 | time: 3.17e-02 |
| epoch:  100 | train acc: 94.8 | loss: 1.93e+01 | valid acc: 93.1 | loss: 8.11e+00 | time: 1.13e-02 |
| epoch:  200 | train acc: 96.5 | loss: 1.17e+01 | valid acc: 94.8 | loss: 7.21e+00 | time: 1.23e-02 |
| epoch:  300 | train acc: 98.3 | loss: 7.69e+00 | valid acc: 94.8 | loss: 6.57e+00 | time: 1.48e-02 |
| epoch:  400 | train acc: 98.3 | loss: 6.55e+00 | valid acc: 94.8 | loss: 6.22e+00 | time: 1.97e-02 |
| epoch:  500 | train acc: 98.7 | loss: 5.23e+00 | valid acc: 94.8 | loss: 5.06e+00 | time: 2.62e-02 |
| epoch:  600 | train acc: 98.7 | loss: 4.53e+00 | valid acc: 96.0 | loss: 4.34e+00 | time: 1.23e-02 |
| epoch:  700 | train acc: 99.0 | loss: 4.33e+00 | valid acc: 96.0 | loss: 4.27e+00 | time: 1.16e-02 |
| epoch:  800 | train acc: 99.0 | loss: 4.20e+00 | valid acc: 96.0 | loss: 4.43e+00 | time: 1.39e-02 |
| epoch:  900 | train acc: 99.0 |

In [23]:
normalPNN1 = torch.nn.Sequential(pnn.pLayer(7,5),pnn.pLayer(5,5),pnn.pLayer(5,3))
optimizer = torch.optim.Adam(normalPNN1.parameters(), lr=config.lr)
normalPNN1

Sequential(
  (0): pLayer()
  (1): pLayer()
  (2): pLayer()
)

In [24]:
normalPNN1, train_losses, valid_losses, train_accs, valid_accs = training.train_nn(normalPNN1,
                                                                                   DataLoader(TensorDataset(X_train1,y_train1), batch_size=E_train1),
                                                                                   DataLoader(TensorDataset(X_valid1,y_valid1), batch_size=E_valid1),
                                                                                   optimizer,
                                                                                   pnn.LossFunction, device)

The ID for this training is 1656941177.
| epoch:    0 | train acc: 33.33 | loss: 8.84e+01 | valid acc: 30.95 | loss: 2.96e+01 | time: 1.58e-02 |
| epoch:  100 | train acc: 92.86 | loss: 1.90e+01 | valid acc: 95.24 | loss: 6.12e+00 | time: 9.20e-03 |
| epoch:  200 | train acc: 94.44 | loss: 7.57e+00 | valid acc: 92.86 | loss: 5.04e+00 | time: 8.18e-03 |
| epoch:  300 | train acc: 96.83 | loss: 6.70e+00 | valid acc: 92.86 | loss: 5.17e+00 | time: 8.00e-03 |
| epoch:  400 | train acc: 96.83 | loss: 6.27e+00 | valid acc: 92.86 | loss: 5.22e+00 | time: 9.32e-03 |
| epoch:  500 | train acc: 96.83 | loss: 5.76e+00 | valid acc: 95.24 | loss: 5.10e+00 | time: 7.55e-03 |
| epoch:  600 | train acc: 97.62 | loss: 5.33e+00 | valid acc: 95.24 | loss: 4.85e+00 | time: 7.61e-03 |
Early stop.
Finished.


In [25]:
normalPNN2 = torch.nn.Sequential(pnn.pLayer(4,5),pnn.pLayer(5,5),pnn.pLayer(5,3))
optimizer = torch.optim.Adam(normalPNN2.parameters(), lr=config.lr)
normalPNN2

Sequential(
  (0): pLayer()
  (1): pLayer()
  (2): pLayer()
)

In [26]:
normalPNN2, train_losses, valid_losses, train_accs, valid_accs = training.train_nn(normalPNN2,
                                                                                   DataLoader(TensorDataset(X_train2,y_train2), batch_size=E_train2),
                                                                                   DataLoader(TensorDataset(X_valid2,y_valid2), batch_size=E_valid2),
                                                                                   optimizer,
                                                                                   pnn.LossFunction, device)

The ID for this training is 1656941184.
| epoch:    0 | train acc: 34.44 | loss: 6.31e+01 | valid acc: 30.00 | loss: 2.12e+01 | time: 1.56e-02 |
| epoch:  100 | train acc: 96.67 | loss: 5.75e+00 | valid acc: 93.33 | loss: 2.19e+00 | time: 8.92e-03 |
| epoch:  200 | train acc: 97.78 | loss: 3.49e+00 | valid acc: 93.33 | loss: 1.94e+00 | time: 7.40e-03 |
| epoch:  300 | train acc: 97.78 | loss: 3.09e+00 | valid acc: 96.67 | loss: 1.72e+00 | time: 1.50e-02 |
| epoch:  400 | train acc: 98.89 | loss: 2.91e+00 | valid acc: 96.67 | loss: 1.64e+00 | time: 8.96e-03 |
| epoch:  500 | train acc: 98.89 | loss: 2.68e+00 | valid acc: 96.67 | loss: 1.35e+00 | time: 7.78e-03 |
| epoch:  600 | train acc: 98.89 | loss: 2.52e+00 | valid acc: 96.67 | loss: 1.29e+00 | time: 6.79e-03 |
| epoch:  700 | train acc: 98.89 | loss: 2.48e+00 | valid acc: 96.67 | loss: 1.39e+00 | time: 9.63e-03 |
| epoch:  800 | train acc: 98.89 | loss: 2.37e+00 | valid acc: 96.67 | loss: 1.51e+00 | time: 6.45e-03 |
| epoch:  900 |